#LEAST SQUARES
Στον αλγόριθμο των ελαχίστων τετραγώνων αρχικά προσθέτουμε έναν bias όρο στα δεδομένα. Στη συνέχεια μετατρέπουμε τα labels σε one-hot μορφή, για να μπορέσουμε να χειριστούμε το πρόβλημα της ταξινόμησης σαν πρόβλημα παλινδρόμησης. Τα βάρη του μοντέλου υπολογίζονται απευθείας μέσω των normal equations, χωρίς κάποια επαναληπτική διαδικασία. Για την πρόβλεψη, υπολογίζουμε τις τιμές που αντιστοιχούν σε κάθε κλάση και επιλέγουμε εκείνη με τη μεγαλύτερη τιμή. Τέλος, αξιολογούμε το μοντέλο χρησιμοποιώντας το accuracy στο test set.

In [3]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from google.colab import drive
import pandas as pd
import torch
drive.mount('/content/drive')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_path = '/content/drive/MyDrive/wine_dataset.csv'

def add_bias(X):
    return np.c_[np.ones(X.shape[0]), X]

def one_hot(y, K):
    Y = np.zeros((len(y), K))
    for i in range(len(y)):
        Y[i, y[i]] = 1
    return Y

def fit_ls(A, Y, l=0.001):
    I = np.eye(A.shape[1])
    W = np.linalg.solve(np.dot(A.T, A) + l * I, np.dot(A.T, Y))
    return W

def predict_ls(A, W):
    scores = np.dot(A, W)
    return np.argmax(scores, axis=1)

def mse_loss(A, W, Y_true):
    # MSE on one-hot targets (Least Squares objective)
    Y_pred = np.dot(A,W)
    return np.mean((Y_pred - Y_true) ** 2)

df = pd.read_csv(file_path)
X = df.drop('target', axis=1).values
y = df['target'].values

K = len(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=12, stratify=y
)

# add bias
X_train_b = add_bias(X_train)
X_test_b  = add_bias(X_test)

# one-hot for training objective
Y_train = one_hot(y_train, K)
Y_test  = one_hot(y_test, K)

# train
W = fit_ls(X_train_b, Y_train, l=0.001)

# predictions
train_pred = predict_ls(X_train_b, W)
test_pred  = predict_ls(X_test_b, W)

train_acc = (train_pred == y_train).mean()
test_acc  = (test_pred == y_test).mean()

train_mse = mse_loss(X_train_b, W, Y_train)
test_mse  = mse_loss(X_test_b, W, Y_test)

print("--- Least Squares ---")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Test Accuracy : {test_acc:.4f}")
print(f"Train MSE Loss: {train_mse:.6f}")
print(f"Test MSE Loss: {test_mse:.6f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- Least Squares ---
Train Accuracy: 1.0000
Test Accuracy : 1.0000
Train MSE Loss: 0.033128
Test MSE Loss: 0.038622
